# Importando e tratando a base de dados

## Importação dos dados

In [ ]:
import json
import requests
from io import StringIO

# carregando as credenciais
with open("/home/daniel/geohmount/code/credentials.json", "r") as credentials_file:
    credentials = json.load(credentials_file)
# parametros da request (github info)
owner = credentials["github"]["username"]
token = credentials["github"]["token"]
repo = credentials["github"]["repo"]
path = 'trajs/sb/resultado.csv'

# enviando o request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

# convertendo a string em objeto stringIO
string_io_obj = StringIO(r.text)

In [ ]:
import pandas as pd

trajs = pd.read_csv(string_io_obj, sep=";")

In [ ]:
trajs.head(15)

## Tratamento dos dados

Renomeando colunas

In [ ]:
trajs = trajs.rename(columns={'velocidade_km/h': 'velocidade'})

Removendo trajetórias sem campanha

In [ ]:
trajs = trajs.dropna()
trajs

### Criando a coluna categórica dos sentidos das trajetórias

In [ ]:
def graus_em_direcao(graus):
    direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", 
                "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    quociente = graus // 22.5
    resto = graus % 22.5
    if resto > 11.25:
        quociente += 1
    if quociente == 16:
        quociente = 0
    return direcoes[int(quociente)]

In [ ]:
trajs['direcao_cat'] = trajs['direcao_graus'].apply(graus_em_direcao)
trajs.head(10)

### Criando a coluna categórica de velocidades

In [ ]:
def velocidade_em_intervalo(velocidade):
    velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', 
                   '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']
    quociente = velocidade // 5
    return velocidades[int(quociente)]

In [ ]:
trajs['velocidade_cat'] = trajs['velocidade'].apply(velocidade_em_intervalo)
trajs.head(10)

### Criando a coluna categórica de soma de chuva do evento (dados da estação meteorológica)

In [ ]:
def chuva_em_intervalo(chuva):
    chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', 
              '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']
    chuva -= 5
    quociente = chuva // 15
    return chuvas[int(quociente)]

In [ ]:
trajs['evt_sum_rain_cat'] = trajs['evt_sum_rain'].apply(chuva_em_intervalo)
trajs.head(10)

### Tipando a coluna de data

In [ ]:
trajs['data'] = pd.to_datetime(trajs['data'])
trajs.head()

### Criando datasets separados por altitude

In [ ]:
trajs_509 = trajs.query("altitude == 509")
trajs_659 = trajs.query("altitude == 659")
trajs_909 = trajs.query("altitude == 909")

# Criando os gráficos gerais

## Velocidade

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

fig = px.bar_polar(trajs_659, r='velocidade', theta='direcao_cat', color='velocidade_cat', 
                   category_orders={'velocidade_cat': velocidades,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Plasma_r,)

fig.update_layout(legend=dict(
     traceorder='reversed',
     yanchor='top',
     y=0.99,
     xanchor='right',
     x=1.1,
     title_font_size=14),
     legend_title_text='<b>Velocidade',
     title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
            'y': 0.98, 'x': 0.5, 'xref': 'paper'},
     font_family='Open Sans',
     polar_radialaxis_showticklabels=False,
     annotations=[dict(
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.01, y=0.95,
        text='<b>Altitude: 659m',
        font=dict(size=14, color='black'))],
     polar_angularaxis_tickfont_size = 15,
     paper_bgcolor='rgb(255, 255, 255)'
)

fig.update_traces(hovertemplate=
                  '<b>%{theta}</b><br>'+
                  'velocidade (km/h): %{r:.1f}<br>')

fig.show()

## Chuva

In [ ]:
chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']

fig = px.bar_polar(trajs_659, r='evt_sum_rain', theta='direcao_cat', color='evt_sum_rain_cat', 
                   category_orders={'evt_sum_rain_cat': chuvas,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Blues,)

fig.update_layout(legend=dict(
     traceorder='reversed',
     yanchor='top',
     y=0.99,
     xanchor='right',
     x=1.1,
     title_font_size=14),
     legend_title_text='<b>Precipitação',
     title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
            'y': 0.98, 'x': 0.5, 'xref': 'paper'},
     font_family='Open Sans',
     polar_radialaxis_showticklabels=False,
     annotations=[dict(
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.01, y=0.95,
        text='<b>Altitude: 659m',
        font=dict(size=14, color='black'))],
     polar_angularaxis_tickfont_size = 15,
     paper_bgcolor='rgb(255, 255, 255)'
)

fig.update_traces(hovertemplate=
                  '<b>%{theta}</b><br>'+
                  'chuva (mm): %{r:.1f}<br>')

fig.show()

## Estação úmida/seca

In [ ]:
meses_chuvosos = ('Jan', 'Feb', 'Mar', 'Apr', 'Oct', 'Nov', 'Dec')
meses_secos = ('May', 'Jun', 'Jul', 'Aug', 'Sep')

In [ ]:
trajs_659['month'] = trajs_659['data'].dt.strftime('%b')
trajs_659.head()

In [ ]:
periodo_umido = trajs_659.loc[trajs_659.month.isin(meses_chuvosos)]
periodo_umido.shape

In [ ]:
periodo_seco = trajs_659.loc[trajs_659.month.isin(meses_secos)]
periodo_seco.shape

vel_plot function

In [ ]:
def vel_plot(dataframe, title='<strong>Distribuição das massas de ar resultantes em chuva no Soberbo</strong>'):
    fig = px.bar_polar(dataframe, r='velocidade', theta='direcao_cat', color='velocidade_cat', 
                   category_orders={'velocidade_cat': velocidades,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Plasma_r,)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title_font_size=13),
         legend_title_text='<b>Velocidade',
         title={'text': title,
                'y': 0.97, 'x': 0.5, 'xref': 'paper'},
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         annotations=[dict(
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.01, y=0.95,
            text='<b>Altitude: 659m',
            font=dict(size=14, color='black'))],
         polar_angularaxis_tickfont_size = 15,
    )

    fig.update_traces(hovertemplate=
                      '<b>%{theta}</b><br>'+
                      'velocidade (km/h): %{r:.1f}<br>')
    
    return fig

In [ ]:
def chuva_plot(dataframe, title='<strong>Distribuição das massas de ar resultantes em chuva no Soberbo</strong>'):
    fig = px.bar_polar(dataframe, r='evt_sum_rain', theta='direcao_cat', color='evt_sum_rain_cat', 
                       category_orders={'evt_sum_rain_cat': chuvas,
                                        'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Blues,)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title_font_size=14),
         legend_title_text='<b>Precipitação',
         title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
                'y': 0.98, 'x': 0.5, 'xref': 'paper'},
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         annotations=[dict(
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.01, y=0.95,
            text='<b>Altitude: 659m',
            font=dict(size=14, color='black'))],
         polar_angularaxis_tickfont_size = 15,
         paper_bgcolor='rgb(255, 255, 255)'
    )

    fig.update_traces(hovertemplate=
                      '<b>%{theta}</b><br>'+
                      'chuva (mm): %{r:.1f}<br>')

    return fig

In [ ]:
fig = vel_plot(periodo_umido, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo<br>(período úmido)</b>')
fig.show()

In [ ]:
fig = vel_plot(periodo_seco, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo<br>(período seco)</b>')
fig.show()

In [ ]:
fig = chuva_plot(periodo_umido, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo<br>(período úmido)</b>')
fig.show()

In [ ]:
fig = chuva_plot(periodo_seco, title='<b>Distribuição das massas de ar resultantes em chuva no Soberbo<br>(período seco)</b>')
fig.show()

## Tentando criar um agrupamento

Experimento para criar um agrupamento do gráfico polar

In [ ]:
trajs_659.head()

In [ ]:
gp_trajs = trajs_659.groupby(['campanha', 'direcao_cat', 'velocidade_cat']).mean()
gp_trajs.head()

In [ ]:
def vel_frequency(n):
    return n / gp_trajs['vel_count'].sum() * 100

In [ ]:
gp_trajs['vel_count'] = trajs_659.groupby(['campanha', 'direcao_cat', 'velocidade_cat'])['velocidade_cat'].count()
gp_trajs['vel_frequency'] = gp_trajs['vel_count'].apply(vel_frequency)
gp_trajs

In [ ]:
gp_trajs.reset_index(inplace=True)
gp_trajs

In [ ]:
fig = px.bar_polar(gp_trajs, r='vel_frequency', theta='direcao_cat', color='velocidade_cat', 
                   category_orders={'velocidade_cat': velocidades,
                                    'direcao_cat': direcoes}, 
                   color_discrete_sequence = px.colors.sequential.Plasma_r,)

fig.update_layout(legend=dict(
     traceorder='reversed',
     yanchor='top',
     y=0.99,
     xanchor='right',
     x=1.1,
     title_font_size=14),
     legend_title_text='<b>Velocidade',
     title={'text': '<b>Distribuição das massas de ar resultantes em chuva no Soberbo',
            'y': 0.98, 'x': 0.5, 'xref': 'paper'},
     font_family='Open Sans',
     polar_radialaxis_showticklabels=False,
     annotations=[dict(
        showarrow=False,
        xref='paper',
        yref='paper',
        x=0.01, y=0.95,
        text='<b>Altitude: 659m',
        font=dict(size=14, color='black'))],
     polar_angularaxis_tickfont_size = 15,
     paper_bgcolor='rgb(255, 255, 255)'
)

fig.update_traces(hovertemplate=
                  '<b>%{theta}</b><br>')

fig.show()

Não me parece tão bom, mas talvez tenha melhorado a distinção dos diferentes graus de velocidade

# Criando os gráficos por campanha

## Velocidade

In [ ]:
import plotly.express as px

velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

campanhas = trajs_659['campanha'].unique()

# padronizar as cores nos gráficos pelo valor de velocidade_cat
color_discrete_map = {'0-5 km/h': 'rgb(240, 249, 33)', '5-10 km/h': 'rgb(253, 202, 38)', '10-15 km/h': 'rgb(251, 159, 58)', '15-20 km/h': 'rgb(237, 121, 83)', '20-25 km/h': 'rgb(216, 87, 107)', '25-30 km/h': 'rgb(189, 55, 134)', '30-35 km/h': 'rgb(156, 23, 158)', '35-40 km/h': 'rgb(114, 1, 168)','40-45 km/h': 'rgb(70, 3, 159)'}

for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
        
    fig = px.bar_polar(df, r='velocidade', theta='direcao_cat', color='velocidade_cat', color_discrete_map=color_discrete_map,
                      category_orders={'velocidade_cat': velocidades,
                                      'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Plasma_r)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title=dict(
             text='<b>Velocidade', 
             font={'size': 14})),
         title=f'<b>{campanha}</b>',
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         polar_angularaxis_tickfont_size = 15,
         paper_bgcolor='rgb(255, 255, 255)')

    fig.show()

## Chuva

In [ ]:
import plotly.express as px

chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']
direcoes = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]

campanhas = trajs_659['campanha'].unique()

# padronizar as cores nos gráficos pelo valor de evt_sum_rain_cat
color_discrete_map = {'5-20 mm': 'rgb(247,251,255)', '20-35 mm': 'rgb(222,235,247)', '35-50 mm': 'rgb(198,219,239)', '50-65 mm': 'rgb(158,202,225)', '65-80 mm': 'rgb(107,174,214)', '80-95 mm': 'rgb(66,146,198)', '95-110 mm': 'rgb(33,113,181)', '110-125 mm': 'rgb(8,81,156)', '125-130 mm': 'rgb(8,48,107)'}

for campanha in campanhas:
    df = trajs_659.loc[trajs_659["campanha"] == campanha]
        
    fig = px.bar_polar(df, r='evt_sum_rain', theta='direcao_cat', color='evt_sum_rain_cat', color_discrete_map=color_discrete_map,
                      category_orders={'evt_sum_rain_cat': chuvas,
                                      'direcao_cat': direcoes}, 
                       color_discrete_sequence = px.colors.sequential.Blues)

    fig.update_layout(legend=dict(
         traceorder='reversed',
         yanchor='top',
         y=0.99,
         xanchor='right',
         x=1.1,
         title=dict(
             text='<b>Precipitação', 
             font={'size': 14})),
         title=f'<b>{campanha}</b>',
         font_family='Open Sans',
         polar_radialaxis_showticklabels=False,
         polar_angularaxis_tickfont_size = 15,
         template='plotly_dark')
    
    fig.update_polars()

    fig.show()